# Д/З 2: Регрессия

### Выполнила Елизавета Клыкова, МКЛ221

* Возьмите ноутбук из второго занятия. В нем есть семейство моделей регрессии (Linear, Ridge, Lasso).
* Попробуйте потренировать и провалидировать их на датасете с которым мы работали на занятиии, но в этот раз поменяйте гиперпараметры при обучении и/или предобработку данных (например, можно более точно токенизировать текст, убрать пунктуацию, etc).
* Расскажите, как изменения гиперпараметров и предобработка данных повлияли на метрики (MSE/RMSE/MAE) предсказательной силы ваших моделей.

## Подготовка данных

Перед обучением обучением модели нужно подготовить данные:

- найти\собрать данные
- почистить и предобработать
- преобразовать в матрицы

**Здесь пока повторяется то, что было на занятии**

In [1]:
# импорты необходимых библиотек
import random
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

# import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

In [2]:
# фиксируем seed для воспроизводимости
seed = 117
random.seed(seed)
np.random.seed(seed)

In [3]:
data = pd.read_csv('IMDB-Movie-Data.csv')
print(data.shape)

data.head(3)

(1000, 12)


,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0


#### Удаляем NaN
Заменять на 0 не хочется, поскольку в случае с кассовыми сборами 0 -- это совсем не то же самое, что отсутствие информации.

In [4]:
print(data.isna().any())
data.shape

Rank                  False
Title                 False
Genre                 False
Description           False
Director              False
Actors                False
Year                  False
Runtime (Minutes)     False
Rating                False
Votes                 False
Revenue (Millions)     True
Metascore              True
dtype: bool


(1000, 12)

In [5]:
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
data.shape

(838, 12)

Попробуем предсказывать рейтинг фильма по данным его описания, года, длины в минутах и кассовых сборов.

Колонка "Rating" станет **целевой переменной, или таргетом** (y). Остальные данные будут **обучающей выборкой** (X).

In [6]:
data.Description

0      A group of intergalactic criminals are forced ...
1      Following clues to the origin of mankind, a te...
2      Three girls are kidnapped by a man with a diag...
3      In a city of humanoid animals, a hustling thea...
4      A secret government agency recruits some of th...
                             ...                        
833    While still out to destroy the evil Umbrella C...
834    3 high school seniors throw a birthday party t...
835    Three American college students studying abroa...
836    Romantic sparks occur between two dance studen...
837    A stuffy businessman finds himself trapped ins...
Name: Description, Length: 838, dtype: object

In [7]:
# подготовим описания фильмов
data["text"] = data.Description.apply(lambda x: x.lower().split()) 
data["text"]

0      [a, group, of, intergalactic, criminals, are, ...
1      [following, clues, to, the, origin, of, mankin...
2      [three, girls, are, kidnapped, by, a, man, wit...
3      [in, a, city, of, humanoid, animals,, a, hustl...
4      [a, secret, government, agency, recruits, some...
                             ...                        
833    [while, still, out, to, destroy, the, evil, um...
834    [3, high, school, seniors, throw, a, birthday,...
835    [three, american, college, students, studying,...
836    [romantic, sparks, occur, between, two, dance,...
837    [a, stuffy, businessman, finds, himself, trapp...
Name: text, Length: 838, dtype: object

In [8]:
input_text = list(data.text.values)

In [9]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_text)]
documents[0]

TaggedDocument(words=['a', 'group', 'of', 'intergalactic', 'criminals', 'are', 'forced', 'to', 'work', 'together', 'to', 'stop', 'a', 'fanatical', 'warrior', 'from', 'taking', 'control', 'of', 'the', 'universe.'], tags=[0])

In [10]:
# обучаем модель на текстах описаний фильмов (можно поизменять параметры)
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4, seed=seed)

In [11]:
model.save("D2V.model")  # сохранение модели

In [12]:
# так можно посмотреть на векторы текстов, на которых училась модель
# индекс [] около documents -- это индекс текста из датасета

model.dv[documents[0].tags[0]]

array([-0.2652736 , -0.03069742,  0.10703647, -0.0064543 , -0.07769807],
      dtype=float32)

Теперь нужно добавить векторы в датасет с остальными параметрами.

In [13]:
# создадим список с векторами для каждого текста
vectors = []
for x in documents:
    vec = list(model.dv[x.tags][0])
    vectors.append(vec)

In [14]:
# так получим датафрейм, где все компоненты векторов в отдельных столбцах
split_df = pd.DataFrame(vectors,
                        columns=['v1', 'v2', 'v3','v4',"v5"])

split_df

,v1,v2,v3,v4,v5
0,-0.265274,-0.030697,0.107036,-0.006454,-0.077698
1,-0.259928,0.102542,0.090047,-0.179952,0.021027
2,-0.195645,0.244577,0.094356,-0.577128,-0.227514
3,-0.716626,0.295276,-0.103655,-0.727374,-0.161014
4,-0.392078,0.111106,0.193627,-0.344898,0.007650
...,...,...,...,...,...
833,-0.433192,-0.046895,0.046423,-0.607044,0.016483
834,-0.314163,0.249851,0.036047,-0.614425,-0.247134
835,-0.146316,-0.013046,-0.059592,-0.242093,-0.046573
836,-0.173083,0.256607,0.173760,-0.156157,0.151023


In [15]:
# теперь добавим его к основному датафрейму
result = data.join(split_df, how='left')
result.shape

(838, 18)

In [16]:
result.head(3)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,text,v1,v2,v3,v4,v5
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,"[a, group, of, intergalactic, criminals, are, ...",-0.265274,-0.030697,0.107036,-0.006454,-0.077698
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,"[following, clues, to, the, origin, of, mankin...",-0.259928,0.102542,0.090047,-0.179952,0.021027
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,"[three, girls, are, kidnapped, by, a, man, wit...",-0.195645,0.244577,0.094356,-0.577128,-0.227514


In [17]:
# переопределим датасет, оставив только важное

data_sm = result[['Runtime (Minutes)',"Year",
                  'Rating', 'Votes',
                  'Revenue (Millions)','Metascore',"v1","v2","v3","v4","v5"]
                 ]

data_sm.head()

,Runtime (Minutes),Year,Rating,Votes,Revenue (Millions),Metascore,v1,v2,v3,v4,v5
0,121,2014,8.1,757074,333.13,76.0,-0.265274,-0.030697,0.107036,-0.006454,-0.077698
1,124,2012,7.0,485820,126.46,65.0,-0.259928,0.102542,0.090047,-0.179952,0.021027
2,117,2016,7.3,157606,138.12,62.0,-0.195645,0.244577,0.094356,-0.577128,-0.227514
3,108,2016,7.2,60545,270.32,59.0,-0.716626,0.295276,-0.103655,-0.727374,-0.161014
4,123,2016,6.2,393727,325.02,40.0,-0.392078,0.111106,0.193627,-0.344898,0.007650


## Подготавливаем матрицы

In [18]:
# определяем X и y

X = data_sm.drop(["Rating"],axis=1).values 

display(X, X.shape)

array([[ 1.21000000e+02,  2.01400000e+03,  7.57074000e+05, ...,
         1.07036471e-01, -6.45429874e-03, -7.76980668e-02],
       [ 1.24000000e+02,  2.01200000e+03,  4.85820000e+05, ...,
         9.00470018e-02, -1.79951504e-01,  2.10266951e-02],
       [ 1.17000000e+02,  2.01600000e+03,  1.57606000e+05, ...,
         9.43559036e-02, -5.77127695e-01, -2.27514014e-01],
       ...,
       [ 9.40000000e+01,  2.00700000e+03,  7.31520000e+04, ...,
        -5.95919937e-02, -2.42092997e-01, -4.65725549e-02],
       [ 9.80000000e+01,  2.00800000e+03,  7.06990000e+04, ...,
         1.73760206e-01, -1.56156614e-01,  1.51023403e-01],
       [ 8.70000000e+01,  2.01600000e+03,  1.24350000e+04, ...,
         7.87810236e-02, -1.36689370e-04,  7.89760500e-02]])

(838, 10)

In [19]:
data_sm.isna().any()

Runtime (Minutes)     False
Year                  False
Rating                False
Votes                 False
Revenue (Millions)    False
Metascore             False
v1                    False
v2                    False
v3                    False
v4                    False
v5                    False
dtype: bool

In [20]:
y = data_sm['Rating'].values  # отдельно вынесли массив с рейтингом
y.shape

(838,)

Иногда бывает полезно [нормализовать](https://en.wikipedia.org/wiki/Normalization_(statistics)) данные: это позволяет исправить ситуацию, когда признаки представлены в разных единицах измерения. Для этого используется StandardScaler.

До нормализации:

In [21]:
list(X[0])

[121.0,
 2014.0,
 757074.0,
 333.13,
 76.0,
 -0.2652736008167267,
 -0.030697423964738846,
 0.10703647136688232,
 -0.006454298738390207,
 -0.07769806683063507]

In [22]:
# использзуем стандартизатор
sc = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(sc.fit_transform(X), y, random_state=seed)

После:

In [23]:
list(sc.fit_transform(X)[0])

[0.3446159455904256,
 0.47085817794460877,
 2.921716098869265,
 2.3795765793786647,
 0.9694564665765606,
 0.6518098512257587,
 -1.1728248639062577,
 0.7200068260236252,
 2.2578794219152503,
 0.11296309013198073]

## Обучаем как на паре

In [24]:
# задаем модель регрессора
# силу регуляризации можно варьировать параметром alpha
regressor = Ridge(random_state=seed)

# обучаем
regressor.fit(X_train, y_train)

Ridge(random_state=117)

In [25]:
# давайте предскажем результат для тестовой выборки
y_preds = regressor.predict(X_test)

## Оценка алгоритма

В качестве метрики будем использовать [среднюю абсолютную ошибку](https://www.youtube.com/watch?v=ZejnwbcU8nw). Она показывает отклонение от правильного ответа в тех же единах измерения.

*(а вообще есть [разные способы](https://towardsdatascience.com/what-are-the-best-metrics-to-evaluate-your-regression-model-418ca481755b))*

In [26]:
mean_absolute_error(y_test, y_preds) 

0.3984207190994228

In [27]:
mean_squared_error(y_test, y_preds) 

0.2815074703956998

Попробуйте разные значения для параметра регуляризации alpha при обучении модели. Как они влияют на величину ошибки?

## Обучение моделей с другими гиперпараметрами
Здесь сразу обратимся к гридсерчу, чтобы не перебирать параметры руками.

### Linear

In [28]:
lin_reg = LinearRegression()

linreg_params = {'fit_intercept': [True, False],
                 'positive': [True, False]
                 }

# здесь беру метрику neg_MAE, потому что обычной MAE у grid search нет
grid_search = GridSearchCV(lin_reg,
                           param_grid=linreg_params,
                           scoring='neg_mean_absolute_error',
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Best score: -0.4301980773016873
Best parameters: {'fit_intercept': True, 'positive': False}


Здесь лучшими оказались параметры, которые заложены в модель по дефолту: fit_intercept=True, positive=False. Попробуем Ridge-регуляризацию.

### Ridge

In [29]:
ridge_model = Ridge()

ridge_params = {'alpha': [0.0001, 0.0005, 0.005, 0.01, 0.05, 0.1,
                          0.15, 0.5, 1, 2, 10, 50, 100, 200],
                'fit_intercept': [True, False],
                'solver': ['auto', 'svd', 'cholesky',
                           'lsqr', 'sparse_cg', 'saga'],
                'random_state': [seed]
                }

grid_search = GridSearchCV(ridge_model,
                           param_grid=ridge_params,
                           scoring='neg_mean_absolute_error',
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Best score: -0.42863334389852137
Best parameters: {'alpha': 50, 'fit_intercept': True, 'random_state': 117, 'solver': 'sparse_cg'}


Не помогло: отличие в ~0.002.

### Lasso

In [30]:
lasso_model = Lasso()

lasso_params = {'alpha': [0.0001, 0.0005, 0.005, 0.01, 0.05, 0.1,
                          0.15, 0.5, 1, 2, 10, 50, 100, 200],
                'fit_intercept': [True, False],
                'random_state': [seed],
                'selection': ['cyclic', 'random']}

grid_search = GridSearchCV(lasso_model,
                           param_grid=lasso_params,
                           scoring='neg_mean_absolute_error',
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Best score: -0.42629168810419527
Best parameters: {'alpha': 0.01, 'fit_intercept': True, 'random_state': 117, 'selection': 'random'}


При фиксированном random seed лучше всего сработала Lasso, однако отличия несущественны. В этом случае регуляризация оказалась не нужна. Обучим модель с выбранными параметрами и оценим на тестовой выборке.

In [31]:
new_regressor = Lasso(alpha=0.01, fit_intercept=True, random_state=seed, selection='random')
new_regressor.fit(X_train, y_train)

y_preds = new_regressor.predict(X_test)

print('MAE', mean_absolute_error(y_test, y_preds))
print('MSE', mean_squared_error(y_test, y_preds))

new_regressor.score(X_test, y_test)

MAE 0.4036729748511162
MSE 0.2869694374834664


0.6248994564966357

### Другая токенизация + лемматизация

Давайте попробуем другую предобработку: токенизацию и лемматизацию nltk с удалением пунктуации и стоп-слов.

In [32]:
import string
from tqdm.auto import tqdm

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('punkt')

punct = string.punctuation
stopws = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [33]:
def lemmatize_with_nltk(text, lemmatize=True, punct=punct, stopws=stopws):
    words = [w for w in word_tokenize(text.lower().strip()) if w not in punct]
    if lemmatize:
        return [lemmatizer.lemmatize(w) for w in words if w not in stopws]
    return words

In [34]:
def prepare_df(filename='IMDB-Movie-Data.csv', lemmatize=True):
    data = pd.read_csv(filename)
    data.dropna(inplace=True)
    data.reset_index(drop=True, inplace=True)
    data['text'] = data.Description.apply(lambda x: lemmatize_with_nltk(x, lemmatize))
    data.drop(columns=['Rank', 'Title', 'Genre', 'Description', 'Director', 'Actors'],
              inplace=True)
    return data

In [35]:
def make_vectors(data, vector_size, window, min_count, seed=seed):

    texts = list(data.text.values)
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]
    # тут будем менять параметры
    model = Doc2Vec(documents,
                    vector_size=vector_size, window=window,
                    min_count=min_count, workers=4, seed=seed)
    vectors = [list(model.dv[doc.tags][0]) for doc in documents]
    vec_cols = ['v' + str(i) for i in range(vector_size)]
    vec_df = pd.DataFrame(vectors, columns=vec_cols)

    return data.join(vec_df, how='left')

In [36]:
df = prepare_df()
vectorized = make_vectors(df, vector_size=5, window=2, min_count=1)
vectorized.head()

,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,text,v0,v1,v2,v3,v4
0,2014,121,8.1,757074,333.13,76.0,"[group, intergalactic, criminal, forced, work,...",-0.114046,-0.073360,0.105861,0.156401,-0.035035
1,2012,124,7.0,485820,126.46,65.0,"[following, clue, origin, mankind, team, find,...",-0.017048,0.034024,0.079850,0.119554,0.088964
2,2016,117,7.3,157606,138.12,62.0,"[three, girl, kidnapped, man, diagnosed, 23, d...",0.103961,0.159609,0.079953,-0.212059,-0.145194
3,2016,108,7.2,60545,270.32,59.0,"[city, humanoid, animal, hustling, theater, im...",-0.206329,0.135011,-0.127786,-0.104821,-0.021514
4,2016,123,6.2,393727,325.02,40.0,"[secret, government, agency, recruit, dangerou...",0.016859,-0.020678,0.172728,0.155322,0.111348


In [37]:
def prepare_for_training(vec_df, seed=seed):
    X = vec_df.drop(['Rating', 'text'], axis=1).values 
    y = vec_df['Rating'].values

    sc = StandardScaler()
    X_train, X_test, y_train, y_test = train_test_split(sc.fit_transform(X),
                                                        y, random_state=seed)

    return X_train, X_test, y_train, y_test

In [38]:
X_train, X_test, y_train, y_test = prepare_for_training(vectorized)

Попробуем гридсерч для модели с регуляризацией лассо еще раз.

In [39]:
lasso_model = Lasso()

lasso_params = {'alpha': [0.0001, 0.0005, 0.005, 0.01, 0.05, 0.1,
                          0.15, 0.5, 1, 2, 10, 50, 100, 200],
                'fit_intercept': [True, False],
                'random_state': [seed],
                'selection': ['cyclic', 'random']}

grid_search = GridSearchCV(lasso_model,
                           param_grid=lasso_params,
                           scoring='neg_mean_absolute_error',
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Best score: -0.4272079566826263
Best parameters: {'alpha': 0.01, 'fit_intercept': True, 'random_state': 117, 'selection': 'cyclic'}


Заметим, что лучшим опять оказался параметр alpha=0.01.

In [40]:
def train_and_evaluate_lasso(X_train, X_test, y_train, y_test):
    new_regressor = Lasso(alpha=0.01, fit_intercept=True,
                          random_state=seed, selection='cyclic')
    new_regressor.fit(X_train, y_train)
    y_preds = new_regressor.predict(X_test)

    mae = mean_absolute_error(y_test, y_preds)
    mse = mean_squared_error(y_test, y_preds)
    score = new_regressor.score(X_test, y_test)

#     print('MAE', mae)
#     print('MSE', mse)
#     print('Score', score)

    return mae, mse, score

In [41]:
train_and_evaluate_lasso(X_train, X_test, y_train, y_test)

(0.40070805498111295, 0.28455928687920407, 0.6280497877985043)

Это немного помогло: MAE/MSE чуть-чуть уменьшились, а score вырос с 0.624 до 0.628 (конечно, это мелочи). Попробуем теперь поэкспериметировать с параметрами Doc2Vec.

In [42]:
def optimize_vector_params(vec_sizes=[3, 4, 5, 7, 10],
                           windows=[1, 2, 3, 4, 5],
                           min_freqs=[1, 2, 3, 4, 5],
                           lemmatize=True):

    best_size, best_window, best_freq, best_score = 0, 0, 0, 0
    best_mae, best_mse = 10000, 10000

    for vec_size in tqdm(vec_sizes):
        for window in windows:
            for min_freq in min_freqs:
                # print('Testing vec_size {}, window {}, min_freq {}...'.format(vec_size, window, min_freq))
                X_train, X_test, y_train, y_test = prepare_for_training(
                    make_vectors(prepare_df(lemmatize=lemmatize), vec_size, window, min_freq))
                mae, mse, score = train_and_evaluate_lasso(
                    X_train, X_test, y_train, y_test)
                if mae < best_mae:
                    best_mae = round(mae, 3)
                    best_mse = round(mse, 3)
                    best_score = round(score, 3)
                    best_size = vec_size
                    best_window = window
                    best_freq = min_freq
                # print()

    return best_mae, best_mse, best_score, best_size, best_window, best_freq

In [43]:
best_mae, best_mse, best_score, best_size, best_window, best_freq = optimize_vector_params()
print('Best results: MAE {}, MSE {}, score {} with vector size {}, window {} and min frequency {}.'.format(
    best_mae, best_mse, best_score, best_size, best_window, best_freq))

  0%|          | 0/5 [00:00<?, ?it/s]

Best results: MAE 0.399, MSE 0.287, score 0.624 with vector size 4, window 1 and min frequency 1.


Грустно, что это не сработало :( Что, если отбросить идею с лемматизацией и вернуться предобработке с простой токенизацией, но повторить эксперимент с векторами?

In [44]:
best_mae, best_mse, best_score, best_size, best_window, best_freq = optimize_vector_params(lemmatize=False)
print('Best results: MAE {}, MSE {}, score {} with vector size {}, window {} and min frequency {}.'.format(
    best_mae, best_mse, best_score, best_size, best_window, best_freq))

  0%|          | 0/5 [00:00<?, ?it/s]

Best results: MAE 0.398, MSE 0.286, score 0.626 with vector size 4, window 4 and min frequency 5.


Примерно то же самое, но хотя бы не хуже... Можно еще поменять векторизацию на Count/TfidfVectorizer или поиграться с параметром alpha у Doc2Vec, но уже поздно и я хочу спать :(